In [2]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
import pickle

import ipynb_path
now_file_name = ipynb_path.get().split('/')[-1].replace('.ipynb','')

input_path = '../input/'
status_file_name = 'status.csv'
station_file_name = 'station.csv'
trip_file_name = 'trip.csv'
weather_file_name = 'weather.csv'
trip_start_date_file_name = 'trip_start_date.csv'
trip_end_date_file_name = 'trip_end_date.csv'
train_sta_wea_tri_end_file_name = 'train_all_row.csv'
train_sta_wea_notna_all_file_name = 'all_row_notna.csv'

output_path = '../output/'
model_path = '../model/'

In [3]:
# #実行したらコメントアウトする
# #保存ファイルの重複が起こる可能性がある
# ###########################################################
# # もう一度notebookを動かす際はモデルリストのstart_numとend_numを
# # 設定してください
# ###########################################################
# st_in = input('初めの数字を入力してください')
# end_in = input('終わりの数字を入力してください')
# start_num = int(st_in)
# end_num = int(end_in)

# model_name_list = [f'model_{i}_{now_file_name}.sav' for i in range(start_num, end_num)]
# submit_file_name_list = [f'submission_{i}_{now_file_name}.csv' for i in range(start_num,end_num)]
# model_submit_dict = dict(zip(model_name_list,submit_file_name_list))

初めの数字を入力してください 2
終わりの数字を入力してください 22


In [16]:
# ##train_all_rowを作る作業

# #データを読み込む作業
# status = pd.read_csv(input_path + status_file_name)
# station = pd.read_csv(input_path + station_file_name)
# weather = pd.read_csv(input_path + weather_file_name)
# trip = pd.read_csv(input_path + trip_file_name)

# #datatime型に直す
# #マジで時間長い　注意
# trip['start_date'] = pd.to_datetime(trip['start_date'])
# trip['end_date'] = pd.to_datetime(trip['end_date'])


# #時刻を切り上げる
# trip['start_date'] = trip['start_date'].dt.ceil("H")
# trip['end_date'] = trip['end_date'].dt.ceil("H")

# #tripでフラグを作るための前準備
# trip_start_date = trip[['duration', 'start_date', 'start_station_id', 'subscription_type']]
# trip_end_date = trip[['duration', 'end_date', 'end_station_id', 'subscription_type']]

# #dateとhourを分離する
# trip_start_date['hour'] = trip_start_date['start_date'].dt.hour
# trip_start_date['date'] = trip_start_date['start_date'].dt.floor("D")

# #subscription_typeをダミー変数化する
# trip_start_date['subscription_type'] = trip_start_date['subscription_type'].replace('Customer',1).replace('Subscriber',2)

# #カウントエンコーダでカウントしたい
# from category_encoders.count import CountEncoder
# ce = CountEncoder(cols=['start_date'])
# ce.fit(trip_start_date)
# trip_start_date = ce.transform(trip_start_date)

# #列名を変更する
# trip_start_date = trip_start_date.rename(columns={'start_station_id':'station_id', 'duration':'start_duration', 'subscription_type_Subscriber':'start_subscription_type_Subscriber'})

# # trip_start_date.to_csv(input_path + trip_start_date_file_name, index=None)


# #trip_end_dateについて！
# #dateとhourを分離する
# trip_end_date['hour'] = trip_end_date['end_date'].dt.hour
# trip_end_date['date'] = trip_end_date['end_date'].dt.floor("D")

# #subscription_typeをダミー変数化する
# trip_end_date['subscription_type'] = trip_end_date['subscription_type'].replace('Customer',1).replace('Subscriber',2)

# #でカウントしたい
# from category_encoders.count import CountEncoder
# ce = CountEncoder(cols=['end_date'])
# ce.fit(trip_end_date)
# trip_end_date = ce.transform(trip_end_date)

# #列名を変更する
# trip_end_date = trip_end_date.rename(columns={'end_station_id':'station_id', 'duration':'end_duration', 'subscription_type_Subscriber':'end_subscription_type_Subscriber'})

# # trip_end_date.to_csv(input_path + trip_end_date_file_name, index=None)

In [4]:
trip_start_date_row = pd.read_csv(input_path + trip_start_date_file_name)
trip_end_date_row  = pd.read_csv(input_path + trip_end_date_file_name)

In [5]:
trip_start_date = trip_start_date_row.drop_duplicates(subset=['date', 'hour', 'station_id'])
trip_end_date = trip_end_date_row.drop_duplicates(subset=['date', 'hour', 'station_id'])

In [17]:
# #statusのyear, month, dayを結合してdatetime型に
# status['date'] = status['year'].astype(str) + '/' + status['month'].astype(str).str.zfill(2).astype(str) + '/' + status['day'].astype(str).str.zfill(2).astype(str)
# status['date'] = pd.to_datetime(status['date'])

# #weatherの'date'もdatetime型に
# weather['date'] = pd.to_datetime(weather['date'])

# #statusとweatherをくっつける作業
# train_sta_wea = pd.merge(status,weather, on = 'date', how ='left')

# #stationをくっつける作業
# train_sta_wea = pd.merge(train_sta_wea, station[['station_id', 'lat', 'long', 'dock_count','city']], how = 'left', on='station_id')

In [7]:
trip_start_date['date'] = pd.to_datetime(trip_start_date['date'])

<ipython-input-7-8e4000c207b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_start_date['date'] = pd.to_datetime(trip_start_date['date'])


In [8]:
trip_end_date['date'] = pd.to_datetime(trip_end_date['date'])

<ipython-input-8-4b3f90de4c8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_end_date['date'] = pd.to_datetime(trip_end_date['date'])


In [18]:
train_sta_wea_tri = pd.merge(train_sta_wea, trip_start_date, how='left', on=['station_id','date', 'hour'])

In [19]:

# train_sta_wea_tri_end = pd.merge(train_sta_wea_tri, trip_end_date, how='left', on=['station_id','date', 'hour'])

# train_sta_wea_tri_end[['start_duration','start_date','subscription_type_x','end_duration','end_date','subscription_type_y']] = train_sta_wea_tri_end[['start_duration','start_date','subscription_type_x','end_duration','end_date','subscription_type_y']].fillna(0)

# # #train_all_row.csvを保存する
# # train_sta_wea_tri_end_file_name = 'train_all_row.csv'
# # train_sta_wea_tri_end.to_csv(input_path + train_sta_wea_tri_end_file_name, index=False)

In [25]:
r = pd.read_csv(input_path + train_sta_wea_tri_end_file_name)

In [27]:
r

,id,year,month,day,hour,station_id,bikes_available,predict,date,max_temperature,...,lat,long,dock_count,city,start_duration,start_date,subscription_type_x,end_duration,end_date,subscription_type_y
0,0,2013,9,1,0,0,11.0,0,2013-09-01,81,...,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2013,9,1,1,0,11.0,0,2013-09-01,81,...,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2013,9,1,2,0,11.0,0,2013-09-01,81,...,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2013,9,1,3,0,11.0,0,2013-09-01,81,...,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2013,9,1,4,0,11.0,0,2013-09-01,81,...,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226395,1226395,2015,8,31,19,69,8.0,0,2015-08-31,82,...,37.34273,-121.89562,15,city1,0.0,0.0,0.0,559.0,134.0,2.0
1226396,1226396,2015,8,31,20,69,8.0,0,2015-08-31,82,...,37.34273,-121.89562,15,city1,0.0,0.0,0.0,0.0,0.0,0.0
1226397,1226397,2015,8,31,21,69,8.0,0,2015-08-31,82,...,37.34273,-121.89562,15,city1,0.0,0.0,0.0,0.0,0.0,0.0
1226398,1226398,2015,8,31,22,69,8.0,0,2015-08-31,82,...,37.34273,-121.89562,15,city1,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
train_sta_wea_tri_end

,id,year,month,day,hour,station_id,bikes_available,predict,date,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure,mean_sea_level_pressure,min_sea_level_pressure,max_visibility,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,events,wind_dir_degrees,lat,long,dock_count,city,start_duration,start_date,subscription_type_x,end_duration,end_date,subscription_type_y
0,0,2013,9,1,0,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,NaN,354,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2013,9,1,1,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,NaN,354,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2013,9,1,2,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,NaN,354,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2013,9,1,3,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,NaN,354,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2013,9,1,4,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,NaN,354,37.32973,-121.90178,27,city1,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226395,1226395,2015,8,31,19,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,NaN,6,37.34273,-121.89562,15,city1,0.0,0.0,0.0,559.0,134.0,2.0
1226396,1226396,2015,8,31,20,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,NaN,6,37.34273,-121.89562,15,city1,0.0,0.0,0.0,0.0,0.0,0.0
1226397,1226397,2015,8,31,21,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,NaN,6,37.34273,-121.89562,15,city1,0.0,0.0,0.0,0.0,0.0,0.0
1226398,1226398,2015,8,31,22,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,NaN,6,37.34273,-121.89562,15,city1,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# # all_row_notnaを作る！！


# train_sta_wea = pd.read_csv(input_path + train_sta_wea_tri_end_file_name)

# #曜日を追加するための関数を定義
# def get_weekday_jp(dt):
#     w_list = ['月曜日', '火曜日', '水曜日', '木曜日', '金曜日', '土曜日', '日曜日']
#     return(w_list[dt.weekday()])

# #曜日の名前をつける
# train_sta_wea['date']=pd.to_datetime(train_sta_wea['date'])
# train_sta_wea['weekday'] = train_sta_wea['date'].apply(get_weekday_jp)

# #eventsの欠損値を埋める
# train_sta_wea['events'] = train_sta_wea['events'].fillna('なし')

# #曜日とeventsとcityはまとめてダミー変数にする
# train_sta_wea = pd.get_dummies(train_sta_wea,columns=['weekday', 'events', 'city'])

# # #欠損値のない、加工されたデータを保存
# # train_sta_wea.to_csv(input_path + train_sta_wea_notna_all_file_name,index=False)

In [3]:
train_sta_wea = pd.read_csv(input_path + train_sta_wea_notna_all_file_name)

In [14]:
pd.set_option('display.max_columns',60)
train_sta_wea

,id,year,month,day,hour,station_id,bikes_available,predict,date,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure,mean_sea_level_pressure,min_sea_level_pressure,max_visibility,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,wind_dir_degrees,lat,long,dock_count,start_duration,start_date,subscription_type_x,end_duration,end_date,subscription_type_y,weekday_土曜日,weekday_日曜日,weekday_月曜日,weekday_木曜日,weekday_水曜日,weekday_火曜日,weekday_金曜日,events_Fog,events_Fog-Rain,events_Rain,events_なし,city_city1,city_city2,city_city3,city_city4,city_city5
0,0,2013,9,1,0,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
1,1,2013,9,1,1,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,2,2013,9,1,2,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,3,2013,9,1,3,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
4,4,2013,9,1,4,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226395,1226395,2015,8,31,19,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,6,37.34273,-121.89562,15,0.0,0.0,0.0,559.0,134.0,2.0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
1226396,1226396,2015,8,31,20,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,6,37.34273,-121.89562,15,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
1226397,1226397,2015,8,31,21,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,6,37.34273,-121.89562,15,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
1226398,1226398,2015,8,31,22,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,6,37.34273,-121.89562,15,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0


In [ ]:
train_sta_wea.info()

In [33]:
col_list_tmp = ['year','month','day']

In [ ]:
# class Convert():
#     def __init__(self, df):
#         self.df = df
        
#     def standard_scale(self):
#         from sklearn.preprocessing import StandardScaler
#         ss = StandardScaler()
#         df_ss = ss.fit_transform(self.df)
#         return df_ss

In [38]:
class Numerical_Inversion():
    def __init__(self, df, not_use_col_list):
        self.df = df
        self.not_use_col_list = not_use_col_list
        
        #dataframeを変換するものとしないもので分ける
        self.convert_df = df.drop(not_use_col_list, axis=1)
        self.not_convert_df = df[not_use_col_list]
        
        
    def convert(self, option):
        
        #'SS'は標準化
        if option == 'SS':
            df_converted = self.standard_scale(self.convert_df)
            return df_converted
        
        #'MM'は正規化
        if option == 'MM':
            df_converted = self.minmax_scale(self.convert_df)
            return df_converted
        
        #'RS'はロバスト化
        if option == 'RS':
            df_converted = self.robust_scale(self.convert_df)
            return df_converted
        
        #'LOG'は対数変換
        if option == 'LOG':
            df_converted = self.log_scale(self.convert_df)
            return df_converted
        
        #'YEO'はyeo-johnson変換
        if option == 'YEO':
            df_converted = self.yeo_scale(self.convert_df)
            return df_converted
        
        print('対応するコマンドを入力してください')
        print('["SS", "MM", "RS", "LOG", "YEO"]')
        
    def make_converted_df(self, option):
        df_converted = self.convert(option)
        all_df = pd.concat([df_converted, self.not_convert_df], axis=1)
        return all_df
        
    def yeo_scale(self, df):
        from sklearn.preprocessing import MinMaxScaler, PowerTransformer
        mm = MinMaxScaler()
        pt = PowerTransformer(standardize=False)
        df_mm = mm.fit_transform(df)
        df_pt_row = pt.fit_transform(df_mm)
        df_pt = pd.DataFrame(df_pt_row, columns=df.columns)
        return df_pt
    
    def log_scale(self, df):
        import numpy as np
        df_log_row = df.apply(np.log1p)
        df_log = pd.DataFrame(df_log_row, columns=df.columns)
        return df_log
    
    def robust_scale(self, df):
        from sklearn.preprocessing import RobustScaler
        rs = RobustScaler()
        df_rs_row = rs.fit_transform(df)
        df_rs = pd.DataFrame(df_rs_row, columns=df.columns)
        return df_rs
    
    
    def standard_scale(self, df):
        from sklearn.preprocessing import StandardScaler
        ss = StandardScaler()
        df_ss_row = ss.fit_transform(df)
        df_ss = pd.DataFrame(df_ss_row, columns=df.columns)
        return df_ss
    
    def minmax_scale(self, df):
        from sklearn.preprocessing import MinMaxScaler
        mm = MinMaxScaler()
        df_mm_row = mm.fit_transform(df)
        df_mm = pd.DataFrame(df_mm_row, columns=df.columns)
        return df_mm

In [39]:
not_use_col_list = ['id', 'month', 'day', 'hour', 'bikes_available','predict', 'date' ]
NI = Numerical_Inversion(train_sta_wea,not_use_col_list)

In [40]:
df_ss = NI.make_converted_df('SS')
df_mm = NI.make_converted_df('MM')
df_rs = NI.make_converted_df('RS')
df_log = NI.make_converted_df('LOG')
df_yeo = NI.make_converted_df('YEO')

In [41]:
df_ss.head()

,year,station_id,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure,mean_sea_level_pressure,min_sea_level_pressure,max_visibility,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,wind_dir_degrees,lat,long,dock_count,start_duration,start_date,subscription_type_x,end_duration,end_date,subscription_type_y,weekday_土曜日,weekday_日曜日,weekday_月曜日,weekday_木曜日,weekday_水曜日,weekday_火曜日,weekday_金曜日,events_Fog,events_Fog-Rain,events_Rain,events_なし,city_city1,city_city2,city_city3,city_city4,city_city5,id,month,day,hour,bikes_available,predict,date
0,-1.695875,-1.707481,1.170552,1.125628,1.190974,1.258623,1.18735,1.083253,-0.433431,0.120969,0.050977,-1.068838,-0.623561,-0.896753,0.074227,0.386944,0.5854,-0.375208,-0.304091,-0.162003,-0.406188,0.707313,-1.289582,1.52269,2.346454,-0.017023,-0.420078,-0.535733,-0.016555,-0.412601,-0.528851,-0.407596,2.43975,-0.409878,-0.407596,-0.407596,-0.407596,-0.407596,-0.117851,-0.052414,-0.355729,0.384446,1.837117,-1.0,-0.333333,-0.333333,-0.27735,0,9,1,0,11.0,0,2013-09-01
1,-1.695875,-1.707481,1.170552,1.125628,1.190974,1.258623,1.18735,1.083253,-0.433431,0.120969,0.050977,-1.068838,-0.623561,-0.896753,0.074227,0.386944,0.5854,-0.375208,-0.304091,-0.162003,-0.406188,0.707313,-1.289582,1.52269,2.346454,-0.017023,-0.420078,-0.535733,-0.016555,-0.412601,-0.528851,-0.407596,2.43975,-0.409878,-0.407596,-0.407596,-0.407596,-0.407596,-0.117851,-0.052414,-0.355729,0.384446,1.837117,-1.0,-0.333333,-0.333333,-0.27735,1,9,1,1,11.0,0,2013-09-01
2,-1.695875,-1.707481,1.170552,1.125628,1.190974,1.258623,1.18735,1.083253,-0.433431,0.120969,0.050977,-1.068838,-0.623561,-0.896753,0.074227,0.386944,0.5854,-0.375208,-0.304091,-0.162003,-0.406188,0.707313,-1.289582,1.52269,2.346454,-0.017023,-0.420078,-0.535733,-0.016555,-0.412601,-0.528851,-0.407596,2.43975,-0.409878,-0.407596,-0.407596,-0.407596,-0.407596,-0.117851,-0.052414,-0.355729,0.384446,1.837117,-1.0,-0.333333,-0.333333,-0.27735,2,9,1,2,11.0,0,2013-09-01
3,-1.695875,-1.707481,1.170552,1.125628,1.190974,1.258623,1.18735,1.083253,-0.433431,0.120969,0.050977,-1.068838,-0.623561,-0.896753,0.074227,0.386944,0.5854,-0.375208,-0.304091,-0.162003,-0.406188,0.707313,-1.289582,1.52269,2.346454,-0.017023,-0.420078,-0.535733,-0.016555,-0.412601,-0.528851,-0.407596,2.43975,-0.409878,-0.407596,-0.407596,-0.407596,-0.407596,-0.117851,-0.052414,-0.355729,0.384446,1.837117,-1.0,-0.333333,-0.333333,-0.27735,3,9,1,3,11.0,0,2013-09-01
4,-1.695875,-1.707481,1.170552,1.125628,1.190974,1.258623,1.18735,1.083253,-0.433431,0.120969,0.050977,-1.068838,-0.623561,-0.896753,0.074227,0.386944,0.5854,-0.375208,-0.304091,-0.162003,-0.406188,0.707313,-1.289582,1.52269,2.346454,-0.017023,-0.420078,-0.535733,-0.016555,-0.412601,-0.528851,-0.407596,2.43975,-0.409878,-0.407596,-0.407596,-0.407596,-0.407596,-0.117851,-0.052414,-0.355729,0.384446,1.837117,-1.0,-0.333333,-0.333333,-0.27735,4,9,1,4,11.0,0,2013-09-01


In [42]:
df_mm.head()

,year,station_id,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure,mean_sea_level_pressure,min_sea_level_pressure,max_visibility,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,wind_dir_degrees,lat,long,dock_count,start_duration,start_date,subscription_type_x,end_duration,end_date,subscription_type_y,weekday_土曜日,weekday_日曜日,weekday_月曜日,weekday_木曜日,weekday_水曜日,weekday_火曜日,weekday_金曜日,events_Fog,events_Fog-Rain,events_Rain,events_なし,city_city1,city_city2,city_city3,city_city4,city_city5,id,month,day,hour,bikes_available,predict,date
0,0.0,0.0,0.660377,0.727273,0.868421,0.906977,0.914894,0.884615,0.666667,0.609375,0.493151,0.44898,0.525773,0.494949,1.0,1.0,1.0,0.122807,0.285714,0.0,0.125,0.983287,0.0,0.954893,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,9,1,0,11.0,0,2013-09-01
1,0.0,0.0,0.660377,0.727273,0.868421,0.906977,0.914894,0.884615,0.666667,0.609375,0.493151,0.44898,0.525773,0.494949,1.0,1.0,1.0,0.122807,0.285714,0.0,0.125,0.983287,0.0,0.954893,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1,9,1,1,11.0,0,2013-09-01
2,0.0,0.0,0.660377,0.727273,0.868421,0.906977,0.914894,0.884615,0.666667,0.609375,0.493151,0.44898,0.525773,0.494949,1.0,1.0,1.0,0.122807,0.285714,0.0,0.125,0.983287,0.0,0.954893,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,2,9,1,2,11.0,0,2013-09-01
3,0.0,0.0,0.660377,0.727273,0.868421,0.906977,0.914894,0.884615,0.666667,0.609375,0.493151,0.44898,0.525773,0.494949,1.0,1.0,1.0,0.122807,0.285714,0.0,0.125,0.983287,0.0,0.954893,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,3,9,1,3,11.0,0,2013-09-01
4,0.0,0.0,0.660377,0.727273,0.868421,0.906977,0.914894,0.884615,0.666667,0.609375,0.493151,0.44898,0.525773,0.494949,1.0,1.0,1.0,0.122807,0.285714,0.0,0.125,0.983287,0.0,0.954893,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4,9,1,4,11.0,0,2013-09-01


In [43]:
df_rs.head()

,year,station_id,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure,mean_sea_level_pressure,min_sea_level_pressure,max_visibility,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,wind_dir_degrees,lat,long,dock_count,start_duration,start_date,subscription_type_x,end_duration,end_date,subscription_type_y,weekday_土曜日,weekday_日曜日,weekday_月曜日,weekday_木曜日,weekday_水曜日,weekday_火曜日,weekday_金曜日,events_Fog,events_Fog-Rain,events_Rain,events_なし,city_city1,city_city2,city_city3,city_city4,city_city5,id,month,day,hour,bikes_available,predict,date
0,-1.0,-0.985714,0.909091,0.666667,0.75,0.8,0.818182,0.642857,-0.444444,0.0,0.0,-0.684211,-0.388889,-0.611111,0.0,0.0,0.0,-0.333333,-0.2,0.0,0.0,0.277108,-0.755041,1.266091,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.5,0.0,0.0,0.0,0,9,1,0,11.0,0,2013-09-01
1,-1.0,-0.985714,0.909091,0.666667,0.75,0.8,0.818182,0.642857,-0.444444,0.0,0.0,-0.684211,-0.388889,-0.611111,0.0,0.0,0.0,-0.333333,-0.2,0.0,0.0,0.277108,-0.755041,1.266091,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.5,0.0,0.0,0.0,1,9,1,1,11.0,0,2013-09-01
2,-1.0,-0.985714,0.909091,0.666667,0.75,0.8,0.818182,0.642857,-0.444444,0.0,0.0,-0.684211,-0.388889,-0.611111,0.0,0.0,0.0,-0.333333,-0.2,0.0,0.0,0.277108,-0.755041,1.266091,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.5,0.0,0.0,0.0,2,9,1,2,11.0,0,2013-09-01
3,-1.0,-0.985714,0.909091,0.666667,0.75,0.8,0.818182,0.642857,-0.444444,0.0,0.0,-0.684211,-0.388889,-0.611111,0.0,0.0,0.0,-0.333333,-0.2,0.0,0.0,0.277108,-0.755041,1.266091,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.5,0.0,0.0,0.0,3,9,1,3,11.0,0,2013-09-01
4,-1.0,-0.985714,0.909091,0.666667,0.75,0.8,0.818182,0.642857,-0.444444,0.0,0.0,-0.684211,-0.388889,-0.611111,0.0,0.0,0.0,-0.333333,-0.2,0.0,0.0,0.277108,-0.755041,1.266091,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.5,0.0,0.0,0.0,4,9,1,4,11.0,0,2013-09-01


In [8]:
pd.set_option('display.max_columns', 60)
train_sta_wea.head()

,id,year,month,day,hour,station_id,bikes_available,predict,date,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure,mean_sea_level_pressure,min_sea_level_pressure,max_visibility,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,wind_dir_degrees,lat,long,dock_count,start_duration,start_date,subscription_type_x,end_duration,end_date,subscription_type_y,weekday_土曜日,weekday_日曜日,weekday_月曜日,weekday_木曜日,weekday_水曜日,weekday_火曜日,weekday_金曜日,events_Fog,events_Fog-Rain,events_Rain,events_なし,city_city1,city_city2,city_city3,city_city4,city_city5
0,0,2013,9,1,0,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
1,1,2013,9,1,1,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,2,2013,9,1,2,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,3,2013,9,1,3,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
4,4,2013,9,1,4,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0


In [38]:
train_sta_wea.drop(col_list_tmp, axis=1)

,id,hour,station_id,bikes_available,predict,date,max_temperature,mean_temperature,min_temperature,max_dew_point,mean_dew_point,min_dew_point,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure,mean_sea_level_pressure,min_sea_level_pressure,max_visibility,mean_visibility,min_visibility,max_wind_Speed,mean_wind_speed,precipitation,cloud_cover,wind_dir_degrees,lat,long,dock_count,start_duration,start_date,subscription_type_x,end_duration,end_date,subscription_type_y,weekday_土曜日,weekday_日曜日,weekday_月曜日,weekday_木曜日,weekday_水曜日,weekday_火曜日,weekday_金曜日,events_Fog,events_Fog-Rain,events_Rain,events_なし,city_city1,city_city2,city_city3,city_city4,city_city5
0,0,0,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
1,1,1,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
2,2,2,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
3,3,3,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
4,4,4,0,11.0,0,2013-09-01,81,70,61,62,58,54,80,67,47,29.94,29.95,29.85,10,10,10,14,4,0.0,1,354,37.32973,-121.90178,27,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226395,1226395,19,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,6,37.34273,-121.89562,15,0.0,0.0,0.0,559.0,134.0,2.0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
1226396,1226396,20,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,6,37.34273,-121.89562,15,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
1226397,1226397,21,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,6,37.34273,-121.89562,15,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0
1226398,1226398,22,69,8.0,0,2015-08-31,82,72,61,62,56,52,84,63,42,29.97,29.92,29.86,10,10,10,22,6,0.0,0,6,37.34273,-121.89562,15,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0


In [34]:
train_sta_wea[col_list_tmp]

,year,month,day
0,2013,9,1
1,2013,9,1
2,2013,9,1
3,2013,9,1
4,2013,9,1
...,...,...,...
1226395,2015,8,31
1226396,2015,8,31
1226397,2015,8,31
1226398,2015,8,31


In [27]:
from datetime import date
from dateutil.relativedelta import relativedelta
import lightgbm as lgb
class make_tr_va_te():
    def __init__(self, df, train_end_next_date):
        self.df = df
        self.train_end_next_date = train_end_next_date
        if type(df['date'][0]):
            df['date'] = pd.to_datetime(df['date'])
        
    def make_train_data(self):
        train_all = self.df[self.df['predict'] == 0]
        train = train_all[train_all['date'] < self.train_end_next_date]
        train_notna = train[train['bikes_available'].notna()]
        return train_notna
        
    def make_valid_data(self):
        valid_all = self.df[self.df['predict'] == 0]
        valid = valid_all[(self.train_end_next_date <= valid_all['date']) & (valid_all['date'] < (self.train_end_next_date + relativedelta(months = 1)))]
        valid_notna = valid[valid['bikes_available'].notna()]
        return valid_notna
                                      
    def make_test_data(self):
        test_all = self.df[self.df['predict'] == 1]
        test = test_all[((self.train_end_next_date + relativedelta(months = 1)) <= test_all['date']) & (test_all['date']< (self.train_end_next_date + relativedelta(months = 2)))]
        return test
    
    def model_for_data(self, train, valid):
        tr_X = train.drop(['id','predict','bikes_available','date'],axis=1)
        tr_y = train['bikes_available']
        va_X = valid.drop(['id','predict','bikes_available','date'],axis=1)
        va_y = valid['bikes_available']   
        return tr_X, tr_y, va_X, va_y
    
    def predict_for_data(self, test):
        te_X = test.drop(['id','predict','bikes_available','date'],axis=1)       
        return te_X
    
    def make_fit_model(self, tr_X, tr_y, va_X, va_y ):
        lgb_train = lgb.Dataset(tr_X, tr_y)
        lgb_eval = lgb.Dataset(va_X, va_y)
        params = {
            'task':'train',
            'boosting_type':'gbdt',
            'objective':'regression',
            'metric':{'rmse'},
            'learning_rate':0.01,
            'num_leaves':23,
            'min_data_in_leaf':1,
            'num_iteration': 10000,
            'verbose':0,
            'random_seed':0,
        }

        model = lgb.train(params, lgb_train, valid_sets=[lgb_train, lgb_eval],
                         verbose_eval=100, num_boost_round=10000,
                         early_stopping_rounds=100)
        return model
    
    def predict(self, model_file_name):
        import pickle
        
        train = self.make_train_data()
        valid = self.make_valid_data()
        test = self.make_test_data()
        tr_X, tr_y, va_X, va_y = self.model_for_data(train, valid)
        te_X = self.predict_for_data(test)
        model  = self.make_fit_model(tr_X, tr_y, va_X, va_y )
        pickle.dump(model, open(model_path + model_file_name, 'wb'))
        valid_best_score = model.best_score['valid_1']['rmse']
        y_pred = model.predict(te_X)
        sub_index = test['id']
        sub_df = pd.DataFrame(list(zip(sub_index, y_pred)))
        print('*****')
        print(valid_best_score)
        return sub_df,valid_best_score

In [28]:
def month_range(start, stop, step = relativedelta(months = 1)):
    current = start
    while current < stop:
        yield current
        current += step

In [29]:
from datetime import date
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta
sub_df_all = pd.DataFrame()
valid_score_list = []
#train_end_next_dateにはvalidationの一ヶ月の初めを入れる
for d in month_range(dt(2014,8,1), dt(2015,8,1)):
    make_data = make_tr_va_te(train_sta_wea, d)
    train_notna = make_data.make_train_data()
    valid_notna = make_data.make_valid_data()
    test =make_data.make_test_data()
    tr_X, tr_y, va_X, va_y = make_data.model_for_data(train_notna, valid_notna)
    te_X = make_data.predict_for_data(test)
    model = make_data.make_fit_model(tr_X, tr_y, va_X, va_y)
    model_name = model_name_list[0]
    sub_df, valid_best_score = make_data.predict(model_name)
    sub_df_all = pd.concat([sub_df_all, sub_df])
    valid_score_list.append(valid_best_score)
cv_score = sum(valid_score_list)/len(valid_score_list)
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')
model_name_list.pop(0)

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.041568 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.17969	valid_1's rmse: 3.39773
[200]	training's rmse: 3.04848	valid_1's rmse: 3.27527
[300]	training's rmse: 2.99593	valid_1's rmse: 3.22884
[400]	training's rmse: 2.96406	valid_1's rmse: 3.20517
[500]	training's rmse: 2.94061	valid_1's rmse: 3.19071
[600]	training's rmse: 2.92301	valid_1's rmse: 3.18202
[700]	training's rmse: 2.90663	valid_1's rmse: 3.17696
[800]	training's rmse: 2.89329	valid_1's rmse: 3.17369
[900]	training's rmse: 2.8802	valid_1's rmse: 3.17004
[1000]	training's rmse: 2.86822	valid_1's rmse: 3.1667
[1100]	training's rmse: 2.85753	valid_1's rmse: 3.16443
[1200]	training's rmse: 2.84669	valid_1's rmse: 3.16183
[1300]	training's rmse: 2.83686	valid_1's rmse: 

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.038316 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.17969	valid_1's rmse: 3.39773
[200]	training's rmse: 3.04848	valid_1's rmse: 3.27527
[300]	training's rmse: 2.99593	valid_1's rmse: 3.22884
[400]	training's rmse: 2.96406	valid_1's rmse: 3.20517
[500]	training's rmse: 2.94061	valid_1's rmse: 3.19071
[600]	training's rmse: 2.92301	valid_1's rmse: 3.18202
[700]	training's rmse: 2.90663	valid_1's rmse: 3.17696
[800]	training's rmse: 2.89329	valid_1's rmse: 3.17369
[900]	training's rmse: 2.8802	valid_1's rmse: 3.17004
[1000]	training's rmse: 2.86822	valid_1's rmse: 3.1667
[1100]	training's rmse: 2.85753	valid_1's rmse: 3.16443
[1200]	training's rmse: 2.84669	valid_1's rmse: 3.16183
[1300]	training's rmse: 2.83686	valid_1's rmse: 

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


...

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037985 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33246	valid_1's rmse: 3.49662
[200]	training's rmse: 3.20341	valid_1's rmse: 3.41532
[300]	training's rmse: 3.14774	valid_1's rmse: 3.38087
[400]	training's rmse: 3.11323	valid_1's rmse: 3.35953
[500]	training's rmse: 3.09055	valid_1's rmse: 3.34655
[600]	training's rmse: 3.07169	valid_1's rmse: 3.34086
[700]	training's rmse: 3.05613	valid_1's rmse: 3.33631
[800]	training's rmse: 3.04348	valid_1's rmse: 3.33454
[900]	training's rmse: 3.03151	valid_1's rmse: 3.33372
[1000]	training's rmse: 3.02112	valid_1's rmse: 3.33208
[1100]	training's rmse: 3.01027	valid_1's rmse: 3.32951
[1200]	training's rmse: 3.00077	valid_1's rmse: 3.32805
[1300]	training's rmse: 2.99163	valid_1's rmse

/Users/ken/opt/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.036378 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 3.33246	valid_1's rmse: 3.49662
[200]	training's rmse: 3.20341	valid_1's rmse: 3.41532
[300]	training's rmse: 3.14774	valid_1's rmse: 3.38087
[400]	training's rmse: 3.11323	valid_1's rmse: 3.35953
[500]	training's rmse: 3.09055	valid_1's rmse: 3.34655
[600]	training's rmse: 3.07169	valid_1's rmse: 3.34086
[700]	training's rmse: 3.05613	valid_1's rmse: 3.33631
[800]	training's rmse: 3.04348	valid_1's rmse: 3.33454
[900]	training's rmse: 3.03151	valid_1's rmse: 3.33372
[1000]	training's rmse: 3.02112	valid_1's rmse: 3.33208
[1100]	training's rmse: 3.01027	valid_1's rmse: 3.32951
[1200]	training's rmse: 3.00077	valid_1's rmse: 3.32805
[1300]	training's rmse: 2.99163	valid_1's rmse

'model_2_No_6_my.sav'

In [ ]:
print('***')
print('***')
print(f'CV score is {cv_score}')
print(f'model_name is {model_name}')

In [40]:
# submit_file_name = model_submit_dict[model_name]
# sub_df_all.to_csv(output_path+submit_file_name, index=False, header=False)
# print(submit_file_name)

submission_1_No_6_my.csv


In [41]:
! signate submit --competition-id=550 '/Users/ken/OneDrive - Yokohama City University/活動/data-sci/データ分析遊び/competition/SIGNATE/オペレーション最適化に向けたシェアサイクルの利用予測/output/submission_1_No_6_my.csv'



You have successfully submitted your predictions.We will send you the submission result to your email address.


In [1]:
#特徴量重要度を算出する
cols = list(tr_X.columns)
f_importance = np.array(model.feature_importance())
# f_importance = f_importance / np.sum(f_importance)

df_importance = pd.DataFrame({'feature': cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False)

display(df_importance)

NameError: name 'tr_X' is not defined